In this assignment students need to predict whether a person
makes over 50K per year or not from classic adult dataset using XGBoost.

The description of the dataset is as follows:

Data Set Information:

Extraction was done by Barry Becker from the 1994 Census
database. A set of reasonably clean records was extracted using
the following conditions: ((AAGE&gt;16) &amp;&amp; (AGI&gt;100) &amp;&amp;
(AFNLWGT&gt;1)&amp;&amp; (HRSWK&gt;0))

Attribute Information:

Listing of attributes: &gt;50K, &lt;=50K.

age: continuous.

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov,
Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school,
Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th,
Doctorate, 5th-6th, Preschool.
education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married,
Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-
managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct,
Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv,
Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-
relative,Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico,
Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan,
Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy,
Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France,
Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia,
Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia,
El-Salvador, Trinadad&amp;Tobago, Peru, Hong, Holand-Netherlands.

In [111]:
import numpy as np
import pandas as pd
train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)
test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test' , skiprows = 1, header = None)
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num','marital_status', 'occupation','relationship', 'race', 'sex', 'capital_gain','capital_loss', 'hours_per_week', 'native_country', 'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

In [56]:
train_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [57]:
test_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [58]:
test_set.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
wage_class        0
dtype: int64

In [59]:
test_set['workclass'].unique()

array([' Private', ' Local-gov', ' ?', ' Self-emp-not-inc',
       ' Federal-gov', ' State-gov', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [60]:
train_set['workclass'].unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [61]:
test_set['occupation'].unique()

array([' Machine-op-inspct', ' Farming-fishing', ' Protective-serv', ' ?',
       ' Other-service', ' Prof-specialty', ' Craft-repair',
       ' Adm-clerical', ' Exec-managerial', ' Tech-support', ' Sales',
       ' Priv-house-serv', ' Transport-moving', ' Handlers-cleaners',
       ' Armed-Forces'], dtype=object)

In [62]:
train_set['occupation'].unique()

array([' Adm-clerical', ' Exec-managerial', ' Handlers-cleaners',
       ' Prof-specialty', ' Other-service', ' Sales', ' Craft-repair',
       ' Transport-moving', ' Farming-fishing', ' Machine-op-inspct',
       ' Tech-support', ' ?', ' Protective-serv', ' Armed-Forces',
       ' Priv-house-serv'], dtype=object)

In [63]:
train_set['relationship'].unique()

array([' Not-in-family', ' Husband', ' Wife', ' Own-child', ' Unmarried',
       ' Other-relative'], dtype=object)

In [64]:
test_set['relationship'].unique()

array([' Own-child', ' Husband', ' Not-in-family', ' Unmarried', ' Wife',
       ' Other-relative'], dtype=object)

In [65]:
train_set['education'].unique()

array([' Bachelors', ' HS-grad', ' 11th', ' Masters', ' 9th',
       ' Some-college', ' Assoc-acdm', ' Assoc-voc', ' 7th-8th',
       ' Doctorate', ' Prof-school', ' 5th-6th', ' 10th', ' 1st-4th',
       ' Preschool', ' 12th'], dtype=object)

In [66]:
test_set['education'].unique()

array([' 11th', ' HS-grad', ' Assoc-acdm', ' Some-college', ' 10th',
       ' Prof-school', ' 7th-8th', ' Bachelors', ' Masters', ' Doctorate',
       ' 5th-6th', ' Assoc-voc', ' 9th', ' 12th', ' 1st-4th',
       ' Preschool'], dtype=object)

In [67]:
test_set['capital_gain'].mean()

1081.9051041090843

In [68]:
test_set['capital_gain'].min()

0

In [69]:
train_set['capital_gain'].max()

99999

In [70]:
test_set[test_set['native_country'] == ' ?']

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
19,40,Private,85019,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,45,?,>50K.
65,41,Private,109912,Bachelors,13,Never-married,Other-service,Not-in-family,White,Female,0,0,40,?,<=50K.
83,44,Self-emp-inc,223881,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,99999,0,50,?,>50K.
188,34,State-gov,513100,Bachelors,13,Married-spouse-absent,Farming-fishing,Not-in-family,Black,Male,0,0,40,?,<=50K.
253,42,Federal-gov,177937,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,40,?,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15684,46,Private,31411,11th,7,Married-civ-spouse,Other-service,Wife,White,Female,0,0,40,?,<=50K.
15711,24,Private,127159,Some-college,10,Never-married,Other-service,Other-relative,White,Female,0,0,24,?,<=50K.
16043,24,State-gov,161783,Bachelors,13,Never-married,Transport-moving,Not-in-family,Black,Male,0,0,40,?,<=50K.
16090,34,Private,143776,Masters,14,Never-married,Prof-specialty,Not-in-family,Black,Male,0,0,45,?,>50K.


In [74]:
train_set.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
wage_class        object
dtype: object

In [75]:
## segregate numerical and categorical columns
numerical_columns_train = train_set.columns[train_set.dtypes!='object']
categorical_columns_train = train_set.columns[train_set.dtypes=='object']
numerical_columns_test = test_set.columns[test_set.dtypes!='object']
categorical_columns_test = test_set.columns[test_set.dtypes=='object']

In [76]:
categorical_columns_train

Index(['workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'native_country', 'wage_class'],
      dtype='object')

In [77]:
numerical_columns_train

Index(['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week'],
      dtype='object')

In [78]:
train_set['workclass'].unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [79]:
train_set['education'].unique()

array([' Bachelors', ' HS-grad', ' 11th', ' Masters', ' 9th',
       ' Some-college', ' Assoc-acdm', ' Assoc-voc', ' 7th-8th',
       ' Doctorate', ' Prof-school', ' 5th-6th', ' 10th', ' 1st-4th',
       ' Preschool', ' 12th'], dtype=object)

In [80]:
train_set['marital_status'].unique()

array([' Never-married', ' Married-civ-spouse', ' Divorced',
       ' Married-spouse-absent', ' Separated', ' Married-AF-spouse',
       ' Widowed'], dtype=object)

In [81]:
train_set['occupation'].unique()

array([' Adm-clerical', ' Exec-managerial', ' Handlers-cleaners',
       ' Prof-specialty', ' Other-service', ' Sales', ' Craft-repair',
       ' Transport-moving', ' Farming-fishing', ' Machine-op-inspct',
       ' Tech-support', ' ?', ' Protective-serv', ' Armed-Forces',
       ' Priv-house-serv'], dtype=object)

In [82]:
train_set['relationship'].unique()

array([' Not-in-family', ' Husband', ' Wife', ' Own-child', ' Unmarried',
       ' Other-relative'], dtype=object)

In [83]:
train_set['race'].unique()

array([' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo',
       ' Other'], dtype=object)

In [84]:
train_set['sex'].unique()

array([' Male', ' Female'], dtype=object)

In [85]:
train_set['native_country'].unique()

array([' United-States', ' Cuba', ' Jamaica', ' India', ' ?', ' Mexico',
       ' South', ' Puerto-Rico', ' Honduras', ' England', ' Canada',
       ' Germany', ' Iran', ' Philippines', ' Italy', ' Poland',
       ' Columbia', ' Cambodia', ' Thailand', ' Ecuador', ' Laos',
       ' Taiwan', ' Haiti', ' Portugal', ' Dominican-Republic',
       ' El-Salvador', ' France', ' Guatemala', ' China', ' Japan',
       ' Yugoslavia', ' Peru', ' Outlying-US(Guam-USVI-etc)', ' Scotland',
       ' Trinadad&Tobago', ' Greece', ' Nicaragua', ' Vietnam', ' Hong',
       ' Ireland', ' Hungary', ' Holand-Netherlands'], dtype=object)

In [86]:
train_set['wage_class'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [112]:
train_set['workclass'] = train_set['workclass'].replace(' ?',train_set['workclass'].mode()[0])
train_set['occupation'] = train_set['occupation'].replace(' ?',train_set['occupation'].mode()[0])
train_set['native_country'] = train_set['native_country'].replace(' ?',train_set['native_country'].mode()[0])

In [113]:
native_country_map = {' United-States':1, ' Cuba':2, ' Jamaica':3, ' India':4, ' Mexico':5,
       ' South':6, ' Puerto-Rico':7, ' Honduras':8, ' England':9, ' Canada':10,
       ' Germany':11, ' Iran':12, ' Philippines':13, ' Italy':14, ' Poland':15,
       ' Columbia':16, ' Cambodia':17, ' Thailand':18, ' Ecuador':19, ' Laos':20,
       ' Taiwan':21, ' Haiti':22, ' Portugal':23, ' Dominican-Republic':24,
       ' El-Salvador':25, ' France':26, ' Guatemala':27, ' China':28, ' Japan':29,
       ' Yugoslavia':30, ' Peru':31, ' Outlying-US(Guam-USVI-etc)':32, ' Scotland':33,
       ' Trinadad&Tobago':34, ' Greece':35, ' Nicaragua':36, ' Vietnam':37, ' Hong':38,
       ' Ireland':39, ' Hungary':40, ' Holand-Netherlands':41}
workclass_map = {' State-gov': 1, ' Self-emp-not-inc': 2, ' Private': 3, ' Federal-gov': 4,
       ' Local-gov': 5,' Self-emp-inc': 6, ' Without-pay': 7,
       ' Never-worked': 8}
wage_class_map = {' <=50K':0, ' >50K':1}
sex_map = {' Male': 1, ' Female': 2}
race_map = {' White': 1, ' Black': 2, ' Asian-Pac-Islander': 3, ' Amer-Indian-Eskimo': 4,
       ' Other': 5}
relationship_map = {' Not-in-family': 1, ' Husband': 2, ' Wife': 3, ' Own-child': 4, ' Unmarried': 5,
       ' Other-relative': 6}
occupation_map = {' Adm-clerical': 1, ' Exec-managerial': 2, ' Handlers-cleaners': 3,
       ' Prof-specialty': 4, ' Other-service': 5, ' Sales': 6, ' Craft-repair': 7,
       ' Transport-moving': 8, ' Farming-fishing': 9, ' Machine-op-inspct': 10,
       ' Tech-support': 11, ' Protective-serv': 12, ' Armed-Forces': 13,
       ' Priv-house-serv': 14}
marital_status_map = {' Never-married': 1, ' Married-civ-spouse': 2, ' Divorced': 3,
       ' Married-spouse-absent': 4, ' Separated': 5, ' Married-AF-spouse': 6,
       ' Widowed': 7}
education_map = {' Bachelors': 1, ' HS-grad': 2, ' 11th': 3, ' Masters': 4, ' 9th': 5,
       ' Some-college': 6, ' Assoc-acdm': 7, ' Assoc-voc': 8, ' 7th-8th': 9,
       ' Doctorate': 10, ' Prof-school': 11, ' 5th-6th': 12, ' 10th': 13, ' 1st-4th': 14,
       ' Preschool': 15, ' 12th': 16}

In [114]:
# # Training set
# train_set['workclass'] = train_set['workclass'].replace(workclass_map)
# train_set['native_country'] = train_set['native_country'].replace(native_country_map)
train_set['wage_class'] = train_set['wage_class'].replace(wage_class_map)
# train_set['sex'] = train_set['sex'].replace(sex_map)
# train_set['race'] = train_set['race'].replace(race_map)
# train_set['relationship'] = train_set['relationship'].replace(relationship_map)
# train_set['occupation'] = train_set['occupation'].replace(occupation_map)
# train_set['marital_status'] = train_set['marital_status'].replace(marital_status_map)
# train_set['education'] = train_set['education'].replace(education_map)
# # Testing Set
# test_set['workclass'] = test_set['workclass'].replace(workclass_map)
# test_set['native_country'] = test_set['native_country'].replace(native_country_map)
test_set['wage_class'] = test_set['wage_class'].replace({' <=50K.': 0,' >50K.': 1})
# test_set['sex'] = test_set['sex'].replace(sex_map)
# test_set['race'] = test_set['race'].replace(race_map)
# test_set['relationship'] = test_set['relationship'].replace(relationship_map)
# test_set['occupation'] = test_set['occupation'].replace(occupation_map)
# test_set['marital_status'] = test_set['marital_status'].replace(marital_status_map)
# test_set['education'] = test_set['education'].replace(education_map)

In [115]:
train_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [117]:
test_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0


In [116]:
X_train = train_set.drop(['wage_class'],axis=1)
y_train = train_set.wage_class
X_test = test_set.drop(['wage_class'],axis=1)
y_test = test_set.wage_class

In [122]:
train_set['workclass'] = train_set['workclass'].astype(str)
train_set['occupation'] = train_set['occupation'].astype(str)
train_set['native_country'] = train_set['native_country'].astype(str)

test_set['workclass'] = test_set['workclass'].astype(str)
test_set['occupation'] = test_set['occupation'].astype(str)
test_set['native_country'] = test_set['native_country'].astype(str)

In [119]:
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier

estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBClassifier(random_state=8)) # can customize objective function with the objective parameter
]
pipe = Pipeline(steps=estimators)
pipe

Pipeline(steps=[('encoder', TargetEncoder()),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=8, ...))])

In [120]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

search_space = {
    'clf__max_depth': [2,8],
    'clf__learning_rate': [0.001, 1.0],
    'clf__subsample': [0.5, 1.0],
    'clf__colsample_bytree': [0.5, 1.0],
    'clf__colsample_bylevel': [0.5, 1.0],
    'clf__colsample_bynode' : [0.5, 1.0],
    'clf__reg_alpha': [0.0, 10.0],
    'clf__reg_lambda': [0.0, 10.0],
    'clf__gamma': [0.0, 10.0]
}

opt = RandomizedSearchCV(pipe, search_space, cv=3, n_iter=10, scoring='roc_auc', random_state=8)
# in reality, you may consider setting cv and n_iter to higher values

In [108]:
opt

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('encoder', TargetEncoder()),
                                             ('clf',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            early_stopping_rounds=None,
                                                            enable_categorical=False,
                                                            eval_metric=None,
                                                            feature_types=None,
                                                            gamma=None,
                                                            gpu_id=None,
                                                            grow_policy=None,
                                                            importance_t...
                                                            predictor=None,
                                                            random_state=8, ...))]),
                   param_distributions={'clf__colsample_bylevel': [0.5, 1.0],
                                        'clf__colsample_bynode': [0.5, 1.0],
                                        'clf__colsample_bytree': [0.5, 1.0],
                                        'clf__gamma': [0.0, 10.0],
                                        'clf__learning_rate': [0.001, 1.0],
                                        'clf__max_depth': [2, 8],
                                        'clf__reg_alpha': [0.0, 10.0],
                                        'clf__reg_lambda': [0.0, 10.0],
                                        'clf__subsample': [0.5, 1.0]},
                   random_state=8, scoring='roc_auc')

In [123]:
opt.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('encoder', TargetEncoder()),
                                             ('clf',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            early_stopping_rounds=None,
                                                            enable_categorical=False,
                                                            eval_metric=None,
                                                            feature_types=None,
                                                            gamma=None,
                                                            gpu_id=None,
                                                            grow_policy=None,
                                                            importance_t...
                                                            predictor=None,
                                                            random_state=8, ...))]),
                   param_distributions={'clf__colsample_bylevel': [0.5, 1.0],
                                        'clf__colsample_bynode': [0.5, 1.0],
                                        'clf__colsample_bytree': [0.5, 1.0],
                                        'clf__gamma': [0.0, 10.0],
                                        'clf__learning_rate': [0.001, 1.0],
                                        'clf__max_depth': [2, 8],
                                        'clf__reg_alpha': [0.0, 10.0],
                                        'clf__reg_lambda': [0.0, 10.0],
                                        'clf__subsample': [0.5, 1.0]},
                   random_state=8, scoring='roc_auc')

In [124]:
opt.score(X_test,y_test)

0.9206584901916383

In [126]:
y_pred = opt.predict(X_test.head(1))
y_pred

array([0])

In [131]:
y_pred = opt.predict(X_test[3:4])
y_pred

array([1])